# Hyperparameter

- 알고리즘을 이용하여 모델링할 때 최적화하기 위한 옵션이다.
- KNN과 Decision Tree는 k값이 중요한데, 이 값을 잘 예측하기 위해서 Random Search, Grid Search를 사용해서 찾아 낼 수 있다.

> Random Search 는 선언하는 범위에서 임의의 개수를 골라 최선의 파라미터 값을 찾는다. 많은 데이터에서 사용하기 좋으며, 단점으로는 최선의 성능인 파라미터 값을 찾아내지 못 할 수도 있다.

> Grid Search 는 우리가 선언한 파라미터 범위를 모두 돌려서 최적의 값을 찾아낸다. 또한 정해진 수치 사이에서 값을 찾아 낼 수 있다. 시간이 많이 걸리는 단점이 있고, 수치를 지정해서 찾을 경우 정해지지 않은 값에서 더 좋은 파라미터가 나올 수도 있다.

## 1. Random Search

In [4]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/income_happy.csv'
data = pd.read_csv(path)

In [6]:
# target 확인
target = 'happiness'

# 데이터 분리
x = data.drop(target, axis=1)
y = data[target]

In [7]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2022)

### 모델링

In [8]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, r2_score

model = DecisionTreeRegressor(max_depth=5, random_state=2022)

model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print(mean_absolute_error(y_test,y_pred)) # mean_absolute_error
print(r2_score(y_test, y_pred)) #R2 SCORE

0.6072155238529675
0.7169483336378495


### 성능튜닝

- 성능을 튜닝할 때, 기본모델: 기본 알고리즘을 사용하는 튜닝 대상 모델 / 최종 모델: RandomizedSearchCV 알고리즘을 사용하는 모델을 선언한다

In [9]:
# 파라미터 선언
param = {'max_depth':range(1,51)}

In [10]:
# 불러오기
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV

In [11]:
#선언하기
model_dst = DecisionTreeRegressor(random_state=2022)

In [12]:
#random_search 선언
model = RandomizedSearchCV(model_dst, param, cv=10, n_iter=20, scoring='r2')

In [13]:
#학습하기
model.fit(x_train, y_train)

RandomizedSearchCV(cv=10, estimator=DecisionTreeRegressor(random_state=2022),
                   n_iter=20, param_distributions={'max_depth': range(1, 51)},
                   scoring='r2')

In [14]:
#정보확인
print('최적파라미터:', model.best_params_)
print('최고성능:', model.best_score_)

최적파라미터: {'max_depth': 5}
최고성능: 0.6117509619179523


In [15]:
#예측하기
y_pred = model.predict(x_test)

In [17]:
#평가하기
print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

0.6072155238529675
0.7169483336378495


## 2.Grid Search

- 그리드 서치에서는 n_iter 파라미터가 들어가지 않는다. 그 이유는 모든 데이터를 가지고 최적의 값을 찾아내기 때문에 무의미하다. 그렇기 때문에 random search에서 n_iter 값을 max로 주게되면 Grid Search의 최적값이 같게 나온다.

In [18]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/admission_simple.csv'
data = pd.read_csv(path)

In [19]:
# 가변수화
cols = ['RANK']
data = pd.get_dummies(data, columns=cols, drop_first=True)

In [20]:
# target 확인
target = 'ADMIT'

# 데이터 분리
x = data.drop(target, axis = 1)
y = data[target] 

In [21]:
# 정규화
cols = ['GRE', 'TOEFL', 'SOP', 'LOR', 'GPA']
x[cols] = (x[cols] - x[cols].min()) / (x[cols].max() - x[cols].min())

# 확인
x.head()

,GRE,TOEFL,SOP,LOR,GPA,RESEARCH,RANK_2,RANK_3,RANK_4,RANK_5
0,0.94,0.928571,0.875,0.875,0.913462,1,0,0,1,0
1,0.68,0.535714,0.750,0.875,0.663462,1,0,0,1,0
2,0.52,0.428571,0.500,0.625,0.384615,1,0,1,0,0
3,0.64,0.642857,0.625,0.375,0.599359,1,0,1,0,0
4,0.48,0.392857,0.250,0.500,0.451923,0,1,0,0,0


In [22]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 7:3으로 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2022)

### 모델링

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

model = KNeighborsClassifier(n_neighbors=5)

model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[75  8]
 [10 57]]
              precision    recall  f1-score   support

           0       0.88      0.90      0.89        83
           1       0.88      0.85      0.86        67

    accuracy                           0.88       150
   macro avg       0.88      0.88      0.88       150
weighted avg       0.88      0.88      0.88       150



### 성능튜닝

In [25]:
#파라미터 선언
param = {'n_neighbors':range(5,51)}

In [26]:
#불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [29]:
#선언하기
model_knn = KNeighborsClassifier()

model = GridSearchCV(model_knn, param, cv=10, scoring='accuracy', n_jobs=-1)

In [30]:
#학습하기
model.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': range(5, 51)}, scoring='accuracy')

In [31]:
#성능확인
print(model.best_params_)
print(model.best_score_)

{'n_neighbors': 32}
0.8457142857142855


In [33]:
#예측하기
y_pred = model.predict(x_test)

In [34]:
#평가하기
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test, y_pred))

[[76  7]
 [11 56]]
              precision    recall  f1-score   support

           0       0.87      0.92      0.89        83
           1       0.89      0.84      0.86        67

    accuracy                           0.88       150
   macro avg       0.88      0.88      0.88       150
weighted avg       0.88      0.88      0.88       150

